* Recommendation System using Alternating Least Square algorithm with Pyspark

Courtesy: https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/master/movie_recommender/movie_recommendation_using_ALS.ipynb

Blog Post: https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1


## Imports

In [47]:
import os
import time
from IPython.core.display import display, HTML
import findspark
findspark.init()
# spark imports
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.types import StringType, ArrayType
from pyspark.ml.recommendation import ALS
from pyspark import SQLContext
from IPython.display import Image
# data science imports
import requests, json
import numpy as np
import pandas as pd

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt


%matplotlib inline

## Create Spark Session

In [2]:
# Create Spark Session
spark=SparkSession.builder.appName("ALS_Movie_Recommendor")\
                          .config("spark.executor.memory", "16g") \
                          .config("spark.master", "local[*]") \
                          .getOrCreate()
# get the spark context
sc=spark.sparkContext
sc.setCheckpointDir('checkpoint')

## Read the data as RDD

In [3]:
data_path=os.path.abspath("./small-data/")
movies=spark.read.load(os.path.join(data_path,"movies.csv"),format='csv', header=True, inferSchema=True)
ratings=spark.read.load(os.path.join(data_path,"ratings.csv"),format='csv', header=True, inferSchema=True)
links=spark.read.load(os.path.join(data_path,"links.csv"),format='csv', header=True, inferSchema=True)
tags=spark.read.load(os.path.join(data_path,"tags.csv"),format='csv', header=True, inferSchema=True)

##  Peek into data

In [107]:
display(HTML("<h3>Movies</h3>"))
movies.show(3)
display(HTML("<h3>Ratings</h3>"))
ratings.show(3)
display(HTML("<h3>Links</h3>"))
links.show(3)
display(HTML("<h3>Tags</h3>"))
tags.show(3)


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
+------+-------+------+---------+
only showing top 3 rows



+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
+-------+------+------+
only showing top 3 rows



+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
+------+-------+---------------+----------+
only showing top 3 rows



## Sparsity of ratings matrix

\begin{align*}
\text{Sparsity}&=1-\frac{\text{Actual number of ratings in matrix}}{\text{(Number of Users) x (Number of Ratings)}}
\end{align*}

In [108]:
numerator=ratings.count()
distinct_users=ratings.select('userId').distinct().count()
distinct_movies=ratings.select('movieId').distinct().count()
denominator=distinct_users*distinct_movies

sparsity=1-(numerator*1.0/denominator)

print(sparsity)

0.9830003169443864


## Splitting data

In [6]:
# converting RDD to pyspark.sql.dataframe.DataFrame
test=ratings.toDF(*ratings.columns)

(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

## Creating ALS model

In [7]:
import time
start_time=time.time()

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, coldStartStrategy="drop")
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [12,13,14]  ) \
            .addGrid(als.maxIter, [18,19,20]) \
            .addGrid(als.regParam, [.17, .18, .19]) \
            .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

train.checkpoint()

models=cv.fit(train)
best_model=models.bestModel

print("--- %s seconds ---" % (time.time() - start_time))

Num models to be tested:  27
--- 560.3325228691101 seconds ---


## Best Model


In [8]:
# Print best_model
#print(best_model)
print(best_model._java_obj.parent().getRank())

# Complete the code below to extract the ALS model parameters
print("**Best Model**")


# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

14
**Best Model**
  Rank: 14
  MaxIter: 18
  RegParam: 0.17


## Predictions

In [9]:
predictions=best_model.transform(test)
predictions=predictions.na.fill(0)
rmse=evaluator.evaluate(predictions)
print(rmse)


0.867480767564069


## Get Predictions for all users

In [22]:
sqlContext=SQLContext(sc)
recommendations=best_model.recommendForAllUsers(15)
def get_recommendation_for_user(recs):
    recs=recs.select("recommendations.movieId","recommendations.rating")
    movies=recs.select("movieId").toPandas().iloc[0,0]
    ratings=recs.select("rating").toPandas().iloc[0,0]
    ratings_matrix=pd.DataFrame(movies,columns=["movieId"])
    ratings_matrix["ratings"]=ratings
    rating_matrix_ps=sqlContext.createDataFrame(ratings_matrix)
    return rating_matrix_ps

## Check predictions and Actual values for top 15 movies of user 101

In [104]:
display(HTML("<h3>Actual Ratings for user 101 </h3>"))
actuals=movies.join(ratings,["movieId"],"left").filter(ratings.userId==104).sort("rating",ascending=False).select("movieId","rating","title","genres").limit(15)
actuals.show()

display(HTML("<h3>Suggestions for user 101 </h3>"))
recos=get_recommendation_for_user(recommendations.filter(recommendations.userId==104)).sort("ratings",ascending=False)
recos.join(movies,["movieId"],"left").show()

+-------+------+--------------------+--------------------+
|movieId|rating|               title|              genres|
+-------+------+--------------------+--------------------+
|    595|   5.0|Beauty and the Be...|Animation|Childre...|
|    357|   5.0|Four Weddings and...|      Comedy|Romance|
|    745|   5.0|Wallace & Gromit:...|Animation|Childre...|
|   1148|   5.0|Wallace & Gromit:...|Animation|Childre...|
|   2572|   5.0|10 Things I Hate ...|      Comedy|Romance|
|   2671|   5.0| Notting Hill (1999)|      Comedy|Romance|
|   4308|   5.0| Moulin Rouge (2001)|Drama|Musical|Rom...|
|   4973|   5.0|Amelie (Fabuleux ...|      Comedy|Romance|
|   5991|   5.0|      Chicago (2002)|Comedy|Crime|Dram...|
|   7034|   5.0|Show Me Love (Fuc...|       Drama|Romance|
|  38038|   5.0|Wallace & Gromit ...|Adventure|Animati...|
|   4306|   4.5|        Shrek (2001)|Adventure|Animati...|
|   1380|   4.5|       Grease (1978)|Comedy|Musical|Ro...|
|    471|   4.5|Hudsucker Proxy, ...|              Comed

+-------+------------------+--------------------+--------------------+
|movieId|           ratings|               title|              genres|
+-------+------------------+--------------------+--------------------+
|  93008| 4.787635803222656|Very Potter Seque...|      Comedy|Musical|
|  25906| 4.787635803222656|Mr. Skeffington (...|       Drama|Romance|
|  77846| 4.787635803222656| 12 Angry Men (1997)|         Crime|Drama|
|   7121|4.6906328201293945|   Adam's Rib (1949)|      Comedy|Romance|
|   6201|4.6877946853637695|    Lady Jane (1986)|       Drama|Romance|
|   4495|4.6877946853637695|Crossing Delancey...|      Comedy|Romance|
|    945| 4.547955513000488|      Top Hat (1935)|Comedy|Musical|Ro...|
|  74282| 4.511919975280762|Anne of Green Gab...|Children|Drama|Ro...|
|   3379|4.5118818283081055| On the Beach (1959)|               Drama|
|  87234| 4.507846832275391|    Submarine (2010)|Comedy|Drama|Romance|
|  93988| 4.494832515716553|North & South (2004)|       Drama|Romance|
|   82

## Posters

In [105]:
actuals=actuals.join(links,["movieId"],"left")
recos=recos.join(links,["movieId"],"left")

In [98]:
def print_posters(user_movies):
    imagesList=""
    for movie in user_movies.collect():
        params={'i':"tt"+str(movie.imdbId),'apiKey':'a13a6d6c'}
        response=requests.get(url="http://www.omdbapi.com/",params=params)
        json_data = json.loads(response.text)
        #print(json_data["Response"])
        if json_data['Response']!="False":
            #plt.figure()
            #img=Image(url=json_data['Poster'],width=150,height=225)
            imagesList+="<img style='width: 150px; height: 225px; margin: 5px; float: left; border: 1px solid black;' src='{}' />".format(json_data['Poster'])
    display(HTML(imagesList))

In [106]:
print_posters(actuals)
print_posters(recos)